# Create SHETRAN Raster Data
*Ben Smith | 12/12/2025*

This script is designed to take online downloads and reconfigure them into raster layers that can be used to setup SHETRAN models.

Todo:
- Check whether you are doing the right thing with NA values at the coast (i.e. whether -9999 values are being averaged and causing errors.
- Add the Northern Ireland Data (C:\Users\nbs65\OneDrive - Newcastle University\SHETRAN - National Setup\Data for Northern Ireland\OSNI_OpenData_50m_DTM)
- Add this notebook file to GitHub.
- Run at 100m, 200, and 500m.
Consider the fixes for the catchments that are below sea level (but that may be one for a later script).

### Preamble

In [1]:
import os
import zipfile

import rasterio
from rasterio.merge import merge
import numpy as np

def write_ascii(
        array: np,
        ascii_ouput_path: str,
        xllcorner: float,
        yllcorner: float,
        cellsize: float,
        ncols: int = None,
        nrows: int = None,
        NODATA_value: int = -9999):

        if len(array.shape) > 0:
            nrows, ncols = array.shape

        file_head = "\n".join(
            ["ncols         " + str(ncols),
             "nrows         " + str(nrows),
             "xllcorner     " + str(xllcorner),
             "yllcorner     " + str(yllcorner),
             "cellsize      " + str(cellsize),
             "NODATA_value  " + str(NODATA_value)])

        with open(ascii_ouput_path, 'wb') as output_filepath:
            np.savetxt(fname=output_filepath, X=array,
                       delimiter=' ', newline='\n', fmt='%1.1f', comments="",
                       header=file_head
                       )


def read_ascii_raster(file_path, data_type=int, return_metadata=True):
    """
    Read ascii raster into numpy array, optionally returning headers.
    """
    headers = []
    dc = {}
    with open(file_path, 'r') as fh:
        for i in range(6):
            asc_line = fh.readline()
            headers.append(asc_line.rstrip())
            key, val = asc_line.rstrip().split()
            dc[key] = val
    ncols = int(dc['ncols'])
    nrows = int(dc['nrows'])
    xll = float(dc['xllcorner'])
    yll = float(dc['yllcorner'])
    cellsize = float(dc['cellsize'])
    nodata = float(dc['NODATA_value'])

    arr = np.loadtxt(file_path, dtype=data_type, skiprows=6)

    headers = '\n'.join(headers)
    headers = headers.rstrip()

    if return_metadata:
        return arr, ncols, nrows, xll, yll, cellsize, nodata, headers, dc
    else:
        return arr

# Function for cell aggregation
def cell_reduce(array, block_size, func=np.mean):
    """
    Resample a NumPy array by reducing its resolution using block aggregation.
    Parameters:
    - array: Input NumPy array.
    - block_size: Factor by which to reduce the resolution.
    - func: Aggregation function (e.g., np.mean, np.min, np.max).
    """
    shape = (array.shape[0] // block_size, block_size, array.shape[1] // block_size, block_size,)

    return func(array.reshape(shape), axis=(1, 3))

## Elevation Data

Elevation data for the DEM and minDEM is taken from the OS Terrain 50 dataset. This is free to download:
https://osdatahub.os.uk/downloads/open/Terrain50

This is used to create the DEM and minimum DEM (which is used for rivers).

In [2]:
# The data is within sub-folders, list these:
OS50_zip_path = "I:/SHETRAN_GB_2021/02_Input_Data/National Data Inputs for SHETRAN UK/terr50_gagg_gb/data/"
OS50_zip_folders = os.listdir(OS50_zip_path)

# Setup a new folder to hold the unzipped data:
OS50_unzipped_folder = os.path.join(OS50_zip_path, 'Unzipped_data/')
if not os.path.exists(OS50_unzipped_folder):
    os.mkdir(OS50_unzipped_folder)

# Unzip the data:
for OS50_zip_folder in OS50_zip_folders:
    zip_folders = os.listdir(os.path.join(OS50_zip_path, OS50_zip_folder))
    for zip_folder in zip_folders:
        with zipfile.ZipFile(os.path.join(OS50_zip_path, OS50_zip_folder, zip_folder), 'r') as zip_ref:
            zip_ref.extractall(OS50_unzipped_folder)

Join the elevation rasters into a single file.

In [ ]:
# List all .asc files in the folder
asc_files = [os.path.join(OS50_unzipped_folder, f) for f in os.listdir(OS50_unzipped_folder) if f.endswith('.asc')]

# Open the files using rasterio:
count = 1
raster_list = []
for asc_file in asc_files:
    print(count, "/", len(asc_files))
    raster = rasterio.open(asc_file,)
    raster_list.append(raster)
    count += 1

# Combine (merge) the rasters:
merged_raster, merged_transform = merge(raster_list)

# Close the opened raster files - you may be able to incorporate this into the loop above.
for raster in raster_list:
    raster.close()

# Extract the first raster band and change 0s to -9999:
merged_raster = merged_raster[0]
merged_raster[merged_raster==0] = -9999

# Write the file as an ascii:
write_ascii(
    array=merged_raster,
    ascii_ouput_path=OS50_zip_path + 'National_OS50.asc',
    xllcorner=merged_transform[2],
    yllcorner=merged_transform[5]-(merged_raster.shape[0]*merged_transform[0]),
    cellsize=merged_transform[0],
)


Regrid the elevation rasters to the desired size.

Note that this does assume that the lower left corner of the national OS50 file is at 0,0, easting northing. Check this if you are redoing this work. you can load the header of the file using the following code:
<code>
headers = []
with open(OS50_zip_path + 'National_OS50.asc', 'r') as fh:
for i in range(6):
asc_line = fh.readline()
headers.append(asc_line.rstrip())
headers
</code>

The first stage of this is to ensure that the 50m data is of the same extent as the 1km data. Rows and columns are added to ensure this. This means that the data has an extent that is in 1km, so can be resampled to divisions of this (1km, 500m, 200m, 100m). This may not work if you try other resolutions as, because the calculations will run from the top left, not the bottom left, the resampled dataset may not have llx/lly coordinates of 0,0. Think about this if you want to use other resolutions!

In [ ]:
national_OS50, _, _, _, _, _, _, _, OS50_header = read_ascii_raster(OS50_zip_path + 'National_OS50.asc', data_type=float)

In [3]:
# # If you have not loaded in the dataset (perhaps because you are only testing the code), you can check the dimentions of the 50m dataset using this code:
#
# OS50_header = {}
# with open(OS50_zip_path + 'National_OS50.asc', 'r') as fh:
#     for i in range(6):
#         asc_line = fh.readline()
#         key, val = asc_line.rstrip().split()
#         OS50_header[key] = val
# OS50_header

{'ncols': '13200',
 'nrows': '24600',
 'xllcorner': '0.0',
 'yllcorner': '0.0',
 'cellsize': '50.0',
 'NODATA_value': '-9999.0'}

In [89]:
# Resize the national dataset to match existing SHETRAN inputs:
# Resize the inputs to the desired SHETRAN grid (top right corner should be x: 661000, y: 1241000):
row_difference = ((661*1000) - float(OS50_header['nrows']) * float(OS50_header['cellsize'])) / float(OS50_header['cellsize'])
col_difference = ((1241*1000) - float(OS50_header['ncols']) * float(OS50_header['cellsize'])) / float(OS50_header['cellsize'])

if row_difference>0:
    # Create the rows of -9999
    new_rows = np.full((row_difference, national_OS50.shape[1]), -9999)
    # Add the new rows to the top
    national_OS50 = np.vstack((new_rows, national_OS50))

# repeat for columns:
if row_difference>0:
    new_cols = np.full((national_OS50.shape[0], col_difference), -9999)
    national_OS50 = np.hstack((national_OS50, new_cols))  # Remember that these need adding at the end.

In [ ]:
# Resample the data at the desired resolution:

# Define the block size for aggregation
resolution_input = float(OS50_header['cellsize'])
resolution_output = 100
block_size = int(resolution_output/resolution_input)  # For 50m -> 100m, use a block size of 2

# Resample using the mean
DEM = cell_reduce(national_OS50, block_size, np.mean)

# Resample using the minimum
minDEM = cell_reduce(national_OS50, block_size, np.min)

In [3]:
# Write the file as an ascii:
write_ascii(
    array=DEM,
    ascii_ouput_path=f'{OS50_zip_path}National_OS50_DEM_{resolution_output}m.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=resolution_output
)

# Write the file as an ascii:
write_ascii(
    array=minDEM,
    ascii_ouput_path=f'{OS50_zip_path}National_OS50_minDEM_{resolution_output}m.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=resolution_output
)


2

Next add the Northern Ireland Data